In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import os
import matplotlib.pyplot as plt

In [20]:
block_size=64
batch_size=128
max_iters=1000
eval_iters=50
learning_rate=3e-4
n_embd=384
n_layer=4
n_head=4
dropout=0.2

In [21]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text=f.read()

In [22]:
char=sorted(set(text))
print(' the length of char is {}'.format(len(char)))
print(char)
vocab_size=len(char)

 the length of char is 80
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [23]:
string_to_int={ch:i for i, ch in enumerate(char)}
int_to_string={i:ch for i, ch in enumerate(char)}
encode=lambda s:[string_to_int[c] for c in s]
decode=lambda s:"".join([int_to_string[c] for c in s])

In [24]:
data = torch.tensor(encode(text), dtype=torch.long)

In [25]:
n=int(len(text)*0.8)
train_data=data[:n]
val_data=data[n:]

In [26]:
#Get batch function
def get_batch(split):
    data=train_data if split=='train' else val_data
    ix=torch.randint(len(data)-block_size,(batch_size,))
    #print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


In [27]:
class Head(nn.Module):
    def __init__(self,head_size):
        super().__init__()
        self.key=nn.Linear(n_embd,head_size,bias=False)
        self.query=nn.Linear(n_embd,head_size,bias=False)
        self.value=nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))

        self.dropout=nn.Dropout(dropout)
    def forward(self,x):
        B,T,C=x.shape
        k=self.key(x)
        q=self.query(x)
        
        #Computing Attention score
        wei=q@k.transpose(-2,-1)*k.shape[-1]**-0.5 # the query multiplied with transpose of key *scaling it with 
        #shape of query(q)=(Q,T,hs) and key(k)=(Q,T,hs) when q@trans(k)=> (Q,T,hs)@(Q,hs,T)=(Q,T,T)<=shape pf wei
        wei=wei.masked_fill(self.tril[:T,:T]==0,float('-inf')) #Shape=>(B,T,T) #create a tril matrix and assign value == 0 with negetive infinity  
        wei=F.softmax(wei,dim=-1) #shape=>(B, T, T)
        wei=self.dropout(wei)
        v=self.value(x)
        out=wei @ v   #value is multiplied with attention score
        return out

In [28]:
class FeedForward(nn.Module):
    def __init__(self,n_embd):
        super().__init__() #This line calls the constructor of the parent class (nn.Module) to initialize the module.
        self.net=nn.Sequential(
            nn.Linear(n_embd,4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd,n_embd),
            nn.Dropout(dropout)
        )
    def forward(self,x):
        return self.net(x)

In [29]:
class MultiheadAttention(nn.Module):
    def __init__(self,num_heads,head_size):
        super().__init__() #This line calls the constructor of the parent class (nn.Module) to initialize the module.
        self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj=nn.Linear(head_size*num_heads,n_embd)
        self.dropout=nn.Dropout(dropout)

        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        out=torch.cat([h(x) for h in self.heads],dim=-1)  
        out = self.dropout(self.proj(out))
        return out  



In [30]:
class Block(nn.Module):
    '''Transformer BLock'''
    def __init__(self,n_embd,n_head):
        super().__init__() #This line calls the constructor of the parent class (nn.Module) to initialize the module.
        head_size=n_embd//n_head
        self.sa = MultiheadAttention(n_head, head_size)   #self attention
        self.ffwd = FeedForward(n_embd)   
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self,x):
        y=self.sa(x)             #self attention
        x=self.ln1(x + y)          #Norm 1
        y=self.ffwd(x)           #feed forward
        x=self.ln2(x + y)          #Norm 2
        return x
     

In [31]:
class GptLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__() #This line calls the constructor of the parent class (nn.Module) to initialize the module.
        self.token_embedding_table=nn.Embedding(vocab_size,n_embd)
        self.position_embedding_table=nn.Embedding(block_size,n_embd)
        self.blocks=nn.Sequential(*[Block(n_embd,n_head) for _ in range(n_layer)])
        self.ln_f=nn.LayerNorm(n_embd) #final layer norm
        self.lm_head=nn.Linear(n_embd,vocab_size)

       # self.apply(self.__init__weights)
    
    def __init__weights(self,module):
        if isinstance(module,nn.Linear):
            torch.nn.init.normal_(module.weight,mean=0.0,std=0.0)
        if module.bias is not None:
            torch.nn.init.zeros(module.bias)
        elif isinstance(module,nn.Embedding):
            torch.nn.init.normal_(module.weight,mean=0.0,std=0.0)
    
    def forward(self,index,target=None):
       logit=self.token_embedding_table(index)
       B, T, C=logit.shape
       # index and targets are both (B,T) tensor of integers
       token_emb=self.token_embedding_table(index)
       pos_emb = self.position_embedding_table(torch.arange(T))
       #pos_emb = self.position_embedding_table(torch.arange(T))
       x=token_emb + pos_emb
     
       x=self.blocks(x) #Some error here
       x=self.ln_f(x)
       logit=self.lm_head(x)

       if target==None:
            loss=None
       else:
         #geting shape of logits
         B, T, C=logit.shape
         logit=logit.view(B*T,C)
         target=target.view(B*T)
         loss=F.cross_entropy(logit,target)
        
       return logit, loss
    
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index


In [32]:
@torch.no_grad()
def estimate_loss():
    out = {'model':[]}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
            out['model'].append(model)
        out[split] = losses.mean()
    model.train()
    return out

In [33]:
model=GptLanguageModel(vocab_size)

In [34]:
context=torch.zeros((1,1),dtype=torch.long,)

In [35]:

model_path = 'model-02.pkl'

if os.path.exists(model_path):
    # Load the model if it exists
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    print('Model loaded successfully.')
else:
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    for iter in range(max_iters):
      if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
        xb, yb = get_batch('train')

    # evaluate the loss
        logits, loss = model.forward(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    print(loss.item())

    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    print('Model saved.')

step: 0, train loss: 4.515, val loss: 4.510
step: 50, train loss: 3.538, val loss: 3.552
step: 100, train loss: 3.255, val loss: 3.270
step: 150, train loss: 3.119, val loss: 3.125
step: 200, train loss: 3.054, val loss: 3.067
step: 250, train loss: 3.006, val loss: 3.020
step: 300, train loss: 2.964, val loss: 2.981
step: 350, train loss: 2.928, val loss: 2.954
step: 400, train loss: 2.899, val loss: 2.921
step: 450, train loss: 2.850, val loss: 2.878
step: 500, train loss: 2.824, val loss: 2.857
step: 550, train loss: 2.801, val loss: 2.840
step: 600, train loss: 2.773, val loss: 2.810
step: 650, train loss: 2.765, val loss: 2.803
step: 700, train loss: 2.750, val loss: 2.781
step: 750, train loss: 2.731, val loss: 2.772
step: 800, train loss: 2.727, val loss: 2.767
step: 850, train loss: 2.707, val loss: 2.743
step: 900, train loss: 2.677, val loss: 2.727
step: 950, train loss: 2.684, val loss: 2.719
2.6535394191741943
Model saved.


In [37]:
print(losses)

{'model': [GptLanguageModel(
  (token_embedding_table): Embedding(80, 384)
  (position_embedding_table): Embedding(64, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiheadAttention(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): Linear(in_features=384, out_features=96, bias=False)
            (query): Linear(in_features=384, out_features=96, bias=False)
            (value): Linear(in_features=384, out_features=96, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementw

In [40]:
prompt = 'do you see me'
context = torch.tensor(encode(prompt), dtype=torch.long) #,device=device)
generated_chars = decode(model.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

do you see med led mo, hemebouy he Iery or EekenYere mainggd wthtd oitar9ind Wgandry whithte  cemb wititorogk br 
